In [1]:
import re
import json
from functools import reduce
from datetime import datetime
from collections import defaultdict
from tqdm import tqdm

import pandas as pd

import nltk
from string import punctuation
from pymystem3 import Mystem
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/furiousteabag/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Importing data

In [2]:
resumes = pd.read_csv("./data/resume_to_hackaton.csv", delimiter=";")
vacancies = pd.read_csv("./data/vacancies_to_hackaton.csv", delimiter=";")

vacancies.drop_duplicates(subset=['id'], inplace=True)

In [3]:
display(resumes)

,uuid,position,organization,description,start,end
0,e08588cc-4ad1-45be-ae95-fa3fd31bc32e,заведующая кафе-кладовщик,ООО Мираж,Составление ттк\r Проведение инвентаризаций\r ...,2012-04,2012-11
1,36c13102-4a62-477b-a475-f9a5b1c48218,Заведующий складом,Оконный завод,руководство складом и цехом по производству ко...,2014-02,2015-09
2,36c13102-4a62-477b-a475-f9a5b1c48218,старший кладовщик,"ООО "" Керамика сервис""",Полное ведение складского учета отдельно стоящ...,2012-03,2013-06
3,36c13102-4a62-477b-a475-f9a5b1c48218,бухгалтер складского учета,"Многопрофильный холдинг ""Вероника"" СПБ","Cклад, бухгалтер по учету ТМЦ и ОС, проведение...",2010-12,2011-05
4,85bfe532-1108-4142-bae5-29818bfc70a0,Старший официант,"ДВЕ ПАЛОЧКИ, сеть ресторанов свободного полета",•\tПодготовка ресторана к открытию;\r •\tВстре...,2008-12,2011-02
...,...,...,...,...,...,...
99995,df2f6158-c03b-4545-a4c5-e5bf80fd5a5b,Администратор ресторана,"Евразия, сеть ресторанов и суши-баров",Контроль работы ресторана,2014-06,2014-12
99996,df2f6158-c03b-4545-a4c5-e5bf80fd5a5b,Официант,jazz bar 48 chairs,Обслуживание гостей по всем европейским станда...,2011-06,2014-06
99997,df2f6158-c03b-4545-a4c5-e5bf80fd5a5b,"Суши-повар, официант, администратор",ООО Страйк,На поседней должности-контроль работы ресторана.,2007-04,2011-05
99998,7b0b9de2-d04c-42d5-a59d-6808b5abef8a,Член бригады ресторана,Макдоналдс,"Кассир, повар, грузчик.",2013-10,2014-11


In [4]:
display(vacancies)

,id,name,area.name,city,company.id,company,company_link,publication_date,salary_from,salary_currency,employment.name,schedule.name,experience.name,key_skills,specializations,description
0,17016760,Официант,Санкт-Петербург,Санкт-Петербург,1905558,2Т,https://hh.ru/employer/1905558,2016-06-16T11:22:58+0300,30000.0,RUR,Полная занятость,Полный день,Нет опыта,NaN,22.193 15.313,"Мы ищем активного, жизнерадостного и дружелюбн..."
1,17759717,Бармен-официант,Санкт-Петербург,unknown,2440865,3Ф Групп,https://hh.ru/employer/2440865,2016-07-15T13:00:14+0300,30000.0,RUR,Полная занятость,Полный день,Нет опыта,Ответственность | Материальная ответственность...,22.175 22.193,Обязанности: Приготовление напитков в баре ...
2,17816582,Администратор ресторана,Санкт-Петербург,Санкт-Петербург,1911500,4ЕС,https://hh.ru/employer/1911500,2016-07-20T17:20:48+0300,45000.0,RUR,Полная занятость,Сменный график,От 3 до 6 лет,NaN,22.329,"Ресторан ""Рандеву"" приглашает на работу админи..."
3,17746005,Кладовщик,Санкт-Петербург,посёлок Шушары,141100,4Ф,https://hh.ru/employer/141100,2016-08-13T09:26:06+0300,38000.0,RUR,Полная занятость,Сменный график,От 1 года до 3 лет,NaN,21.275 21.563,Обязанности: Прием товара (Кока-Кола) на скла...
4,17431972,Кладовщик,Санкт-Петербург,поселок Шушары,141100,4Ф,https://hh.ru/employer/141100,2016-07-12T15:22:23+0300,30000.0,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563 21.564,Обязанности: отпуск и прием на складе ТМЦ вед...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32160,26344256,Кладовщик,Санкт-Петербург,Санкт-Петербург,3414,"ТрансМашХолдинг, Группа компаний",https://hh.ru/employer/3414,2018-06-13T19:25:26+0300,NaN,NaN,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563,"ОАО ""Октябрьский электровагоноремонтный завод""..."
32161,26367364,Кондитер в ресторан (частичная занятость),Санкт-Петербург,unknown,3064104,ТРИГГЕР,https://hh.ru/employer/3064104,2018-06-14T17:56:22+0300,29000.0,RUR,Частичная занятость,Сменный график,От 1 года до 3 лет,NaN,22.204 22.353,"Ресторан ""Счастье у Исаакия"" открывает конкурс..."
32162,26373024,Заведующий производством/заведующий продовольс...,Санкт-Петербург,Пушкин,2130686,ФГБУЗ Санкт-Петербургский Дом-пансионат ветера...,https://hh.ru/employer/2130686,2018-06-15T09:56:47+0300,30000.0,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,22.204 18.297,Обязанности: - Руководство деятельностью пищеб...
32163,26431613,Грузчик-комплектовщик,Санкт-Петербург,unknown,66239,"Элизиум, группа компаний",https://hh.ru/employer/66239,2018-06-19T10:45:48+0300,30000.0,RUR,Полная занятость,Полный день,Нет опыта,NaN,29.495 29.545 21.275 21.564 21.136,Обязанности: Перемещение готовой продукции По...


# Analyzing data

In [5]:
klad = vacancies[vacancies["name"] == "Кладовщик"]

In [6]:
klad[klad["description"].str.contains("задачи", flags=re.IGNORECASE)]

,id,name,area.name,city,company.id,company,company_link,publication_date,salary_from,salary_currency,employment.name,schedule.name,experience.name,key_skills,specializations,description
290,17719816,Кладовщик,Санкт-Петербург,unknown,1863695,JACOBS DOUWE EGBERTS (JDE),https://hh.ru/employer/1863695,2016-07-12T13:48:40+0300,NaN,NaN,Полная занятость,Полный день,От 3 до 6 лет,Складской документооборот | Складской Учет | И...,21.563 21.564,Основные задачи: осуществлять руководство раб...
492,16327708,Кладовщик,Санкт-Петербург,unknown,14208,Акваарт,https://hh.ru/employer/14208,2016-08-01T10:38:27+0300,36000.0,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563,Крупной компании требуется Кладовщик. Мы увлеч...
655,17647889,Кладовщик,Санкт-Петербург,unknown,681883,Велесстрой,https://hh.ru/employer/681883,2016-11-24T10:55:21+0300,45000.0,RUR,Полная занятость,Вахтовый метод,От 1 года до 3 лет,MS Office | 1С: Бухгалтерия и склад | 1С: Упра...,21.563 21.564,"ООО ""Велесстрой"" приглашает на работу Кладовщи..."
728,17909436,Кладовщик,Санкт-Петербург,unknown,1978,Группа компаний ТЕЛРОС,https://hh.ru/employer/1978,2016-08-01T12:55:15+0300,NaN,NaN,Полная занятость,Полный день,От 3 до 6 лет,NaN,21.563 21.564,Крупная телекоммуникационная компания пригла...
729,17775765,Кладовщик,Санкт-Петербург,unknown,1978,Группа компаний ТЕЛРОС,https://hh.ru/employer/1978,2016-07-18T11:29:01+0300,NaN,NaN,Полная занятость,Полный день,От 3 до 6 лет,NaN,21.563 21.564,Крупная телекоммуникационная компания пригла...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30446,14390895,Кладовщик,Санкт-Петербург,Санкт-Петербург,566086,Десерт Фентези,https://hh.ru/employer/566086,2015-08-24T13:28:33+0300,40000.0,RUR,Полная занятость,Сменный график,От 1 года до 3 лет,NaN,21.563 21.564,Должностные обязанности: Работа на складе зам...
30766,14088764,Кладовщик,Санкт-Петербург,Санкт-Петербург,24837,НГ-Энерго,https://hh.ru/employer/24837,2015-08-18T09:16:24+0300,NaN,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563 21.564,"Компания ""НГ-Энерго"" приглашает в команду проф..."
31918,19907730,Кладовщик,Санкт-Петербург,unknown,837503,Сварочные Технологии,https://hh.ru/employer/837503,2017-03-13T10:00:59+0300,30000.0,RUR,Полная занятость,Полный день,Нет опыта,Деловая коммуникация | Организаторские навыки ...,21.176 21.275 21.563 21.564 21.136 15.308,Добрый день! Мы - интернет-магазин промышлен...
32007,18856837,Кладовщик,Санкт-Петербург,unknown,197216,ГК «РОСОХРАНА»,https://hh.ru/employer/197216,2016-11-09T17:12:55+0300,NaN,NaN,Проектная работа,Полный день,От 3 до 6 лет,NaN,21.275 21.563,Что мы ждем от кандидата: Знание порядка орга...


# Manually tagging

In [7]:
pd.set_option('display.max_colwidth', None)

In [8]:
ids = [14829806,
       25374959,
       18985243,
       22011507,
       15945853,
       25845683,
       19176449,
       19852081,
       22681923,
       19899949]

In [9]:
vacancies[vacancies['id'].isin(ids)]

,id,name,area.name,city,company.id,company,company_link,publication_date,salary_from,salary_currency,employment.name,schedule.name,experience.name,key_skills,specializations,description
1799,19899949,Кладовщик,Санкт-Петербург,unknown,1947314,ANCOR Consumer Goods,https://hh.ru/employer/1947314,2017-04-04T17:34:37+0300,50000.0,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563,"Наш клиент, крупная торговая компания, открывает вакансию Кладовщик. Обязанности: Руководство складом пищевой продукции (кондитерские изделия) - хранение, погрузка, разгрузка, сбор заказов, формирование заказов по маршрутам и машинам доставки; Организация и контроль бесперебойной работы склада; Руководство персоналом (грузчики); Отчетность. Требования: Высшее образование; Опыт работы на аналогичной должности от 2-х лет; Опыт работы на складе пищевой продукции обязателен; Опытный пользователь Microsoft Office, 1C. Условия: Оформление в соответствии с ТК РФ Работа на юге города"
8580,25374959,Кладовщик,Санкт-Петербург,Москва,600589,Нью Йоркер РУС,https://hh.ru/employer/600589,2018-04-13T09:40:38+0300,NaN,NaN,Полная занятость,Полный день,Нет опыта,NaN,21.563 21.564,"Обязанности: Размещение и хранение товара на складе Учет товарных остатков Проведение инвентаризации Погрузочно-разгрузочные работы Контроль своевременного пополнения товарного запаса в торговом зале Распределение товара на складе Выдача товара в торговый зал Поддержание чистоты и порядка на складе Работа с товаром по предпродажной подготовке (распаковка товара, оформление ценников, развеска товара) Требования: Опыт работы в сфере розничной торговли товарами народного потребления приветствуется Законченное среднее образование Условия: Официальное трудоустройство с соблюдением всех норм ТК РФ Достойная заработная плата и гибкая система премирования по результатам продаж Молодой и дружный коллектив Возможность повышения квалификации ( внутреннее и внешнее обучение) и карьерного роста Полная занятость, полный день Сменный график работы Работа В ТЦ ""Лето"""
9768,22011507,Кладовщик,Санкт-Петербург,поселок Шушары,7348,SMC Pneumatik,https://hh.ru/employer/7348,2017-08-02T13:06:03+0300,NaN,NaN,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563 21.564,"Обязанности: Разбор поставок, сборка, упаковка и подготовка заказов к отправке. Требования: Аналогичный опыт работы на складе от 1 года; Внимательность, аккуратность, ответственность. Условия: Оформление в соответствии с ТК РФ, ""белая"" заработная плата; График работы: 5/2, полный рабочий день с 9.00 до 18.00; Место работы: теплый склад, п. Шушары, 2-й Бадаевский проезд; Развозка или проезд на любой маршрутке от ст.м.""Купчино"", с Витебского пр. до магазина ""Лента""; Социальный пакет (ДМС, НС, доп. отпуск, 100% оплата больничных листов)."
11782,18985243,Кладовщик,Санкт-Петербург,unknown,872828,"БАЛТИЙСКИЙ БЕРЕГ, ТД",https://hh.ru/employer/872828,2017-01-12T17:41:29+0300,40000.0,RUR,Полная занятость,Сменный график,От 1 года до 3 лет,NaN,21.275 21.563,"Компания ""БАЛТИЙСКИЙ БЕРЕГ""- лидер на рынке по производству и реализации рыбной продукции, приглашает в свою команду кладовщика. Обязанности: Приёмка и отгрузка товара со склада; Оформление складских документов (приходные, расходные ордера); Инвентаризация товара на складе; Работа с терминалом сбора данных. Требования: Среднее-специальное или среднее-техническое образование; Опыт работы от двух лет; Знание 1С 8 в конфигурациях УТ или УПП; Наличие действующей медицинской книжки. Мы предлагаем: Работу в стабильной, динамично развивающейся компании; Оформление согласно ТК РФ; Официальную заработную плату 40 000 рублей; График работы сменный день/ночь; Дружный, молодой коллектив; Место работы Горское шоссе (Комплекс Осиновая роща). Развозка от ст. м. Озерки."
12314,19176449,Кладовщик,Санкт-Петербург,unknown,85975,САКС,https://hh.ru/employer/85975,2016-12-12T09:51:57+0300,25000.0,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563 21.564,"Обязанности: прием хранение выдача, отгру

# Splitting description

In [10]:
# Tags by which we split description.
tags = {
    "Обязанности": ["обязанност", "задачи"],
    "Требования": ["требуется", "требовани", "пожелан"],
    "Условия": ["условия", "предлагаем", "предложени"]
}

# Tags which we exclude.
bad_tags = {
    "Обязанности": [],
    "Требования": ["требований", "требованиям", "требования безо", "требования по"],
    "Условия": ["условиями"]
}

In [11]:
def make_list_from_description(string):
    string = string.replace("ст. м. ", "ст м ")
    string = string.replace("м. ", "ст м ")
    string = string.replace("соц. пакет", "соц пакет")
    
    up_regex = '[А-Я][а-я][^А-Я]+?[А-Я]{2,}|[А-Я][а-я][^А-Я]+?[А-Я][^а-я]|[А-Я][а-я][^А-Я]+'
    
    dot_list = list(filter(None, string.split('. ')))
    comma_list = list(filter(None, string.split(';')))
    new_list = dot_list if len(dot_list) > len(comma_list) else comma_list
    
    if (len(new_list) <= 2):
        up_new_list = re.findall(up_regex, string)
        new_list = up_new_list if len(up_new_list) > len(new_list) else new_list
        if (len(new_list) <= 1):
            return [string]
        
    for i in range(len(new_list)):
        ex_list = new_list[i].split('!')
        if (len(ex_list) > 1):
            del new_list[i]
            for e in ex_list:
                new_list.insert(i, e)
                i += 1
        i -= len(ex_list)
        
    return list(filter(lambda x: x != "" and x != " ", new_list))

In [12]:
def get_data_from_description(description, tags, bad_tags):
    """Extracting text by key words.
    
    Args:
        description (string): text we want to extract
                              tags from.
        tags (dict): keys are tags we want to extract;
                     values are lists of words that
                         match this tag.
    
    Returns:
        data_from_description (dict): 
                     keys are tags;
                     values are text that match key.
    """
    
    # Merging all keywords from tags to 1 list.
    keywords = reduce(lambda x, y: x + y, list(tags.values()))
    bad_keywords = reduce(lambda x, y: x + y, list(bad_tags.values()))
    
    # Splitting description by keywords. 
    part = []
    all_parts = []
    for word in description.split():
        if any(subword in word.lower() for subword in keywords) and (not (any(bad_subword in word.lower() for bad_subword in bad_keywords))):
            all_parts.append(part)
            all_parts.append([word])
            part = []
        else:
            part.append(word)
    all_parts.append(part)

    # Merging lists of strings to strings.
    all_strings = []
    for word_list in all_parts:
        all_strings.append(' '.join(word_list))

    # Creating dictionary with keys same as tags keys and
    # values as all strings matching these keys
    # (value is a text that follows current key)
    data_from_description = {}
    for key in tags.keys():
        data_from_description[key] = []
    
    for string in all_strings:
        for key in tags.keys():
            if any(subword in string.lower() for subword in tags[key]) and (not (any(bad_subword in string.lower() for bad_subword in bad_keywords))):
                data_from_description[key] = make_list_from_description(all_strings[all_strings.index(string) + 1])

    return all_strings, data_from_description

In [13]:
def get_json_from_data( id ):
    record = vacancies[vacancies["id"] == int(id)]
    record = record.drop(['id','company.id', 'company_link', 'salary_currency', 'key_skills', 'specializations', 'description'], axis=1)
    dict_record = record.to_dict('records')[0]
    for key in dict_record:
        if key=='city':
            if pd.isnull(dict_record['city']) or (dict_record['city'] == 'unknown'):
                dict_record['city'] = dict_record['area.name']
        else:
            if pd.isnull(dict_record[key]) or (dict_record[key] == 'unknown'):
                dict_record[key] = ''
    dict_record['q0']=dict_record.pop('name')
    dict_record['q1']=dict_record.pop('area.name')
    dict_record['q2']=dict_record.pop('city')
    dict_record['q3']=dict_record.pop('company')
    dict_record['q4']=dict_record.pop('salary_from')
    dict_record['q5']=dict_record.pop('employment.name')
    dict_record['q6']=dict_record.pop('schedule.name')
    dict_record['q7']=dict_record.pop('experience.name')
    dict_record['q8']=dict_record.pop('publication_date')
    match = re.search(r'\d{4}-\d{2}-\d{2}', dict_record['q8'])
    dict_record['q8'] = abs((datetime.strptime("2020-09-20", '%Y-%m-%d').date() - datetime.strptime(match.group(), '%Y-%m-%d').date()).days)
    
    description = list(vacancies[vacancies["id"] == int(id)]["description"])[0]
    all_strings, description_data = get_data_from_description(description, tags, bad_tags)
    dict_record['q9'] = description_data["Требования"]
    dict_record['q10'] = description_data["Обязанности"]
    dict_record['q11'] = description_data["Условия"]
    
    predict_json = {'common': dict_record}
    dict_json = {f'{id}': predict_json}
    json_record = json.dumps(dict_json,  ensure_ascii=False)
    return dict_json

## testing

In [14]:
with open("./prepared_data/storekeeper_labeled.json", "r") as read_file:
    data = json.load(read_file)

In [15]:
print("Количество ключей: " + str(len(data.keys())))

Количество ключей: 28


In [16]:
parsed_data = {}

for id in list(data.keys()):
    print("Id:")
    print(id)
    print("\n")
    
    print("Human processed data:")
   # print(json.dumps(data[id], indent=4, ensure_ascii=False))
    print("\n")
    
    description = list(vacancies[vacancies["id"] == int(id)]["description"])[0]
    all_strings, data_from_description = get_data_from_description(description, tags, bad_tags)
    
    print("Parsed data:")
    parsed_item = get_json_from_data(id)
    parsed_data = {**parsed_data, **parsed_item}
    print(json.dumps(parsed_item, indent=4, ensure_ascii=False))
    print("\n")
    
    print("Meta data:")
    print(all_strings)
    print("\n")
    
    print("Initial data:")
   # print(description)
    print("\n")
    
    print("-------------------------------------------------------------------------")

Id:
25374959


Human processed data:


Parsed data:
{
    "25374959": {
        "common": {
            "q0": "Кладовщик",
            "q1": "Санкт-Петербург",
            "q2": "Москва",
            "q3": "Нью Йоркер РУС",
            "q4": "",
            "q5": "Полная занятость",
            "q6": "Полный день",
            "q7": "Нет опыта",
            "q8": 891,
            "q9": [
                "Опыт работы в сфере розничной торговли товарами народного потребления приветствуется ",
                "Законченное среднее образование"
            ],
            "q10": [
                "Размещение и хранение товара на складе ",
                "Учет товарных остатков ",
                "Проведение инвентаризации ",
                "Погрузочно-разгрузочные работы ",
                "Контроль своевременного пополнения товарного запаса в торговом зале ",
                "Распределение товара на складе ",
                "Выдача товара в торговый зал ",
                "Поддержание чи

## calculating

## drop irrelevant values

# Finding answers to questions

In [17]:
mystem = Mystem()
russian_stopwords = stopwords.words("russian")

def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords
              and token != " "
              and token.strip() not in punctuation]

    text = " ".join(tokens)

    return text

In [18]:
def prepare_questions(questions):
    for key, value in questions.items():
        questions[key] = preprocess_text(value)
    return questions

def prepare_data(data):
    for key, _ in tqdm(data.items()):
        data[key]["common"]["q9"] = list(map(preprocess_text, data[key]["common"]["q9"]))
        data[key]["common"]["q10"] = list(map(preprocess_text, data[key]["common"]["q10"]))
        data[key]["common"]["q11"] = list(map(preprocess_text, data[key]["common"]["q11"]))
    return data

def prepare_tags(tags):
    for key, value in tags.items():
        tags[key] = list(map(preprocess_text, value))
    return tags

## preparing data

In [19]:
questions_common = {
    "q0" : "Название вакансии",
    "q1" : "Регион",
    "q2" : "Город",
    "q3" : "Название компании",
    "q4" : "Зарплата от",
    "q5" : "Тип занятости",
    "q6" : "Тип графика",
    "q7" : "Опыт",
    "q8" : "Время поиска работы",
    "q9" : "Требования",
    "q10" : "Обязанности",
    "q11" : "Условия"
}

questions_special_klad = {
    "qo": "Отрасль",
    "q1": "С какой системой хранения Вы имели опыт работы?",
    "q2": "Опыт работы с программами",
    "q3": "Опыт с инструментарием",
    "q4": "Типы работ"
}

questions_special_tags_klad = {
     "qo": ["производство", "продукты питания", "Одежда", "продовольствие", "продовольственный", "полиграфический", "завод", 'канцелярия', 'упаковка', 'реализация'],
    "q1": ['Адресная', 'Стеллажная', 'безадресная'],
    "q2": ["1С", "Microsoft Office", "ПК", "MC Office", "Excel", "Word", "Outlook", "SAP", "Microsoft", "MS", "программа"],
    "q3": ["штабелер", "рохля", "складской техники", "погрузчик", "электророхля", "транспортным оборудованием", "электропогрузчик", "терминал сбора данных", "размерная сетка товаров"],
    "q4": []
}

questions_special_pogruz = {
    "q0": "Водительское удостоверение",
    "q1": "Удостоверение тракториста-машиниста нового образца",
    "q2": "Удостоверение тракториста-машиниста старого образца",
    "q3": "Сфера деятельности",
    "q4": "Какой техникой вы управляете?",
    "q5": "Максимальная высота поднятия груза",
    "q6": "С какой системой хранения Вы имели опыт работы?",
    "q7": "Удостоверение стропальщика?",
    "q8": "Типы работ",
    "q9": "Возможность провести тест-драйв на собеседовании",
    "q10": "Знание языков",
    "q11": "Командировки",
    "q12": "Станции метро"
}

questions_special_tags_pogruz = {
    "q0": ["водительское", "категории", "водительское удостоверение", "прав водителя"],
    "q1": ["нового образца"],
    "q2": ["старого образца"],
    "q3": ["ритейл", "производство"],
    "q4": ["электромостовым краном", "погрузчика", "автопогрузчика", "штабелера", "электропогрузчика", "ричтрак", "хап"],
    "q5": ["подъемом на высоту", "высота подъема", "высота"],
    "q6": ["адресная", "стеллажная", "системой хранения"],
    "q7": ["удостоверение стропальщика"],
    "q8": [],
    "q9": ["тест-драйв"],
    "q10": ["язык", "английский"],
    "q11": ["командировки"],
    "q12": ["метро", "ст м"],
}

questions_special_oficiant = {
    "qo": "Профессиональное образование в сфере гостеприимства",
    "q1": "Год окончания обучения",
    "q2": "Дополнительные профессиональные навыки",
    "q3": "Знание языков и уровень владения",
    "q4": "С какой посудой работали?"
}

questions_special_tags_oficiant = {
    "qo": ["образование в сфере гостеприимства",],
    "q1": ['год окончания'],
    "q2": ['обучение в школе сомелье'],
    "q3": ['уровень английского', 'native speaker', 'fluent', 'advanced', 'intermediate', 'pre-intermediate', 'навыки английского', 'разговорные навыки английский', ' разговорный английский', 'разговорные навыки', 'разговорный', 
       'английский язык', 'немецкий язык', 'испанский язык'],
    "q4": ['Snow white','кость']
}

In [20]:
# Preparing questions.
questions_special_oficiant_prepared = prepare_questions(questions_special_oficiant)
questions_special_klad_prepared = prepare_questions(questions_special_klad)
questions_special_pogruz_prepared = prepare_questions(questions_special_pogruz)

questions_common_prepared = prepare_questions(questions_common)

In [21]:
# Preparing tags.
questions_special_tags_oficiant_prepared = prepare_tags(questions_special_tags_oficiant)
questions_special_tags_klad_prepared = prepare_tags(questions_special_tags_klad)
questions_special_tags_pogruz_prepared = prepare_tags(questions_special_tags_pogruz)

## finding answers

In [22]:
with open("./prepared_data/data_splitted_description_vacancies_prepared.json", 'r', encoding='utf-8') as f:
    data_prepared = json.load(f)

In [23]:
len(data_prepared)

17958

In [74]:
def get_features_list(data, idx):
    parsed_data_prepared_list = \
    data[str(idx)]["common"]["q9"] + \
    data[str(idx)]["common"]["q10"] + \
    data[str(idx)]["common"]["q11"]
    return parsed_data_prepared_list

In [78]:
def prepare_answers(data, idx):
    
    special_questions_answers = {}

    relevant_vacancies = ["официант", "кладовщик", "погрузчик"]
    vacancy_name = data[str(idx)]["common"]["q0"]
    
    
    parsed_data_list = get_features_list(data, idx)
    
    if relevant_vacancies[0] in vacancy_name.lower():
        
        for question, tags in questions_special_tags_oficiant_prepared.items():
            special_questions_answers[question] = []
            
            for tag in tags:
                for data_item in parsed_data_list:
                    if tag in data_item:
                        special_questions_answers[question].append(data_item)
                        
            special_questions_answers[question] = list(set(special_questions_answers[question]))
        
        
        
        
    elif relevant_vacancies[1] in vacancy_name.lower():
        
        for question, tags in questions_special_tags_klad_prepared.items():
            special_questions_answers[question] = []
            
            for tag in tags:
                for data_item in parsed_data_list:
                    if tag in data_item:
                        special_questions_answers[question].append(data_item)
                        
            special_questions_answers[question] = list(set(special_questions_answers[question]))
            
            
        special_questions_answers["q4"] = data[str(idx)]["common"]["q10"]
            
    elif relevant_vacancies[2] in vacancy_name.lower():
        
        for question, tags in questions_special_tags_pogruz_prepared.items():
            special_questions_answers[question] = []
            
            for tag in tags:
                for data_item in parsed_data_list:
                    if tag in data_item:
                        special_questions_answers[question].append(data_item)
                        
            special_questions_answers[question] = list(set(special_questions_answers[question]))
        
        
        special_questions_answers["q8"] = data[str(idx)]["common"]["q10"]
        
    return special_questions_answers

In [85]:
%%time
for index in tqdm(data_prepared.keys()):
    special_info = prepare_answers(data_prepared, index)
    data_prepared[index]['special'] = special_info
        

100%|██████████| 17958/17958 [00:00<00:00, 25969.43it/s]

CPU times: user 697 ms, sys: 67 µs, total: 697 ms
Wall time: 769 ms


In [87]:
with open('./prepared_data/result.json', 'w', encoding='utf-8') as fp:
    json.dump(data_prepared, fp, ensure_ascii=False, indent=4)